In [1]:
from collections import namedtuple

import re

Nanobot = namedtuple('Nanobot', ('x', 'y', 'z', 'r'))

def get_bots(filename):
    bots = []
    with open(filename) as infile:
        for l in infile:
            x, y, z, r = re.sub(r'[^0-9\-]', ' ', l).split()
            bots.append(Nanobot(int(x), int(y), int(z), int(r)))
    return bots

In [2]:
bots = get_bots('input.txt')
big = sorted(bots, key=lambda b: b.r)[-1]

def dist(a, b):
    return abs(a.x - b.x) + abs(a.y - b.y) + abs(a.z - b.z)

def dist_origin(a):
    return abs(a.x) + abs(a.y) + abs(a.z)

sum(1 for b in bots if dist(big, b) < big.r)

In [3]:
from heapq import heappush, heappop

Coords = namedtuple('Coords', ('x', 'y', 'z'))

class Cube:
    best = None  # current best location among ALL cubes [num_in_range, distance, Coords]
    hq = None  # heap of Cubes that might contain the optimal solution

    root = None  # Coords of corner with lowest x, y, and z
    size = None  # length of each side of the cube
    def_count = None  # number of bots that are definitely in range of this entire cube
    maybe = None  # list of bots that might be in range of parts of this cube but not in "definitely"
    nearest_dist = None  # lowest Manhattan distance of any point in this cube to origin
    max_in_range = None  # max possible bots that might be in range of this cube
    
    def __init__(self, root, size, bots, def_count):
        """
        Args:
            root: Coords of corner with lowest x, y, and z
            size: length of each side of the cube.  should be 2^x
            bots: list of bots to optimize for 
            def_count: number of bots not in bots list that are definitely in range of this entire cube
        """
        self.root = root
        self.size = size
        radius = size // 2
        center = Coords(root.x + radius, root.y + radius, root.z + radius)
        self.maybe = []
        for b in bots:
            d = dist(b, center)
            if d > (b.r + 3 * radius):
                # this cube is completely out of range
                continue
            if d <= (b.r - 3 * radius):
                # all of this cube is in range of b
                def_count += 1
                continue
            self.maybe.append(b)
        nearest_coords = Coords(*(
            min((abs(coord), coord), (abs(coord + size), coord + size))[1]
            for coord in root
        ))
        self.nearest_dist = dist_origin(nearest_coords)
        self.max_in_range = len(self.maybe) + def_count
        self.def_count = def_count
        if self.maybe_best():
            if len(self.maybe) == 0:
                # the best point of this cube is the nearest corner
                self.set_best(self.def_count, self.nearest_dist, nearest_coords)
            else:
                heappush(self.hq, self)
    
    def __lt__(self, other):
        # enable heapq sorting
        return (-self.max_in_range, self.nearest_dist) < (-other.max_in_range, other.nearest_dist)

    def reduce(self):
        if not self.maybe_best():
            # no point in this cube can possibly be better than the current best
            return
        
        # divide up cube into subcubes
        interval = self.size // 2
        for x in range(self.root.x, self.root.x + self.size, interval):
            for y in range(self.root.y, self.root.y + self.size, interval):
                for z in range(self.root.z, self.root.z + self.size, interval):
                    Cube(Coords(x, y, z), interval, self.maybe, self.def_count)

    @classmethod
    def set_best(cls, ct, dst, coords):
        cls.best = (ct, dst, coords)
        print(cls.best)

    @classmethod
    def reset(cls):
        Cube.best = (0, 0, None)
        Cube.hq = []
            
    def maybe_best(self):
        return (self.max_in_range, -self.nearest_dist) > (self.best[0], -self.best[1])
        

In [4]:
from math import ceil, log

bots = get_bots('input.txt')
min_x = min(b.x for b in bots)
min_y = min(b.y for b in bots)
min_z = min(b.z for b in bots)
max_x = max(b.x for b in bots)
max_y = max(b.y for b in bots)
max_z = max(b.z for b in bots)
root = Coords(min_x, min_y, min_z)
size = 2 ** ceil(log(max(max_x - min_x, max_y - min_y, max_z - min_z), 2))

Cube.reset()
main = Cube(root, size, bots, 0)
while Cube.hq:
    heappop(Cube.hq).reduce()

(805, 92889285, Coords(x=17733234, y=41114340, z=34041711))
(962, 93826283, Coords(x=18374826, y=41219170, z=34232287))
(963, 93826284, Coords(x=18374826, y=41219170, z=34232288))
(964, 93826285, Coords(x=18374826, y=41219171, z=34232288))
(965, 93826287, Coords(x=18374826, y=41219173, z=34232288))
(967, 93826287, Coords(x=18374826, y=41219172, z=34232289))
(968, 93826289, Coords(x=18374827, y=41219172, z=34232290))
(969, 93826291, Coords(x=18374828, y=41219173, z=34232290))
(970, 93826293, Coords(x=18374829, y=41219174, z=34232290))
